In [2]:
# data table
import pandas as pd
import os

datadis=pd.read_excel(os.path.join('dataPruebas','dataPeru_indicadores.xlsx'),
                     dtype={'Ubigeo': object})
datadis.head()

,Ubigeo,Departamento,Provincia,Distrito,Poblacion,Superficie,IDH2019,Educ_sec_comp2019_pct,NBI2017_pct,Viv_sin_serv_hig2017_pct
0,010202,Amazonas,Bagua,ARAMANGO,10940,809.07,0.361182,37.545885,62.9,9.1
1,010201,Amazonas,Bagua,BAGUA,26091,150.99,0.584063,70.795501,25.4,0.7
2,010203,Amazonas,Bagua,COPALLIN,6319,99.05,0.467245,52.571361,45.4,2.1
3,010204,Amazonas,Bagua,EL PARCO,1492,18.48,0.489049,70.215409,59.9,1.5
4,010205,Amazonas,Bagua,IMAZA,24323,4430.84,0.298089,31.295719,88.4,11.7


In [5]:
# map

os.environ['USE_PYGEOS'] = '0'

import geopandas as gpd

datadismap=gpd.read_file(os.path.join('mapsPruebas','DistritosMap.zip'))

datadismap.head()

DriverError: '/vsizip/mapsPruebas\DistritosMap.zip' does not exist in the file system, and is not recognized as a supported dataset name.